In [1]:
from collections.abc import Callable
import hashlib
import inspect
import itertools
import os
import warnings

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader, TensorDataset, random_split
import torchvision
from torchvision import datasets
from torchvision.utils import make_grid
from torchvision.transforms import ToTensor, Compose, Resize

from philosofool.torch.nn_models import (
    ResidualBlock, ResidualNetwork, NeuralNetwork,
    Generator, Discriminator,
    compute_convolution_dims, conv_dims_1d
)
from philosofool.torch.nn_loop import (
    TrainingLoop, JSONLogger, CompositeLogger, StandardOutputLogger, GANLoop, TrainingLoop,
    EndOnBatchCallback, SnapshotCallback, VerboseTrainingCallback
)

In [2]:
def show_image(img):
    """Show image implied by input tensor. The input is assumed to be normalized."""
    img = img / 2 + 0.5     # unnormalize
    npimg = img.detach().numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

## NN Models


In [3]:
def read_text(path) -> str:
    with open(path, 'r') as f:
        return f.read()

def get_fashionMINST_data() -> tuple[DataLoader, DataLoader, dict]:

    training_data = datasets.FashionMNIST(
        root='data', train=True, download=True, transform=ToTensor()
    )
    test_data = datasets.FashionMNIST(
        root='data', train=False, download=True, transform=ToTensor()
    )
    batch_size = 64
    train_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
    test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
    classes = {
        0: 'T-shirt/top',
        1: 'Trouser',
        2: 'Pullover',
        3: 'Dress',
        4: 'Coat',
        5: 'Sandal',
        6: 'Shirt',
        7: 'Sneaker',
        8: 'Bag',
        9: 'Ankle'}
    return train_dataloader, test_dataloader, classes

def get_food100_data():
    transform = Compose([Resize((256, 256)), ToTensor()])
    training_data = datasets.Food101(
        root='data', split='train', download=True, transform=transform
    )
    test_data = datasets.Food101(
        root='data', split='test', download=True, transform=transform
    )
    batch_size = 64
    train_dataloader = DataLoader(
        training_data, batch_size=batch_size, shuffle=True, num_workers=8)
    test_dataloader = DataLoader(
        test_data, batch_size=batch_size, shuffle=True, num_workers=8)

    classes_path = os.path.join(os.getcwd(), 'data/food-101/meta/classes.txt')
    classes = dict(enumerate(read_text(classes_path).split('\n')))

    return train_dataloader, test_dataloader, classes


train_dataloader, test_dataloader, classes = get_fashionMINST_data()
# train_dataloader, test_dataloader, classes = get_food100_data()

In [4]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

input_dims = tuple(int(x) for x in X.shape[2:])
in_channels = int(X.shape[1])


Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cpu


In [6]:
loss_fn = nn.CrossEntropyLoss()
configurations = [
    ('baseline', {'n_hidden_layers': 1, 'width': 256}),
    ('h2_w128', {'n_hidden_layers': 2, 'width': 128}),
    ('h3_w94', {'n_hidden_layers': 3, 'width': 96}),
    ('h4_w64', {'n_hidden_layers': 4, 'width': 64})
]

In [7]:
def model_experiment(configurations):
    X, _ = next(iter(test_dataloader))
    input_dims = int(X.shape[2] * X.shape[3])
    input_channels = int(X.shape[1])
    for model_name, config in configurations:
        model = NeuralNetwork(input_dims * input_channels, len(classes), **config)
        model.to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
        logger = CompositeLogger(JSONLogger('data/logs/food100_' + model_name + '.json'), StandardOutputLogger(500))
        training_loop = TrainingLoop(model, optimizer, loss_fn, logger)
        print(model_name)
        training_loop.fit(train_dataloader, test_dataloader, epochs=1)

# model_experiment(configurations)

In [8]:
def _make_model_path(model: nn.Module, model_name: str, directory: str | None) -> str:
    string_hash = hashlib.sha1(bytes(str(model).encode())).hexdigest()
    full_path = model_name + '_' + string_hash+ '.model'
    if directory:
        full_path = os.path.join(directory, full_path)
    return os.path.normpath(full_path)


def save_model(model: nn.Module, model_name: str, directory: str | None):
    """Save model, using string repr to assure that models are not duplicated once trained."""
    full_path = _make_model_path(model, model_name, directory)
    if not os.path.exists(os.path.dirname(full_path)):
        os.makedirs(os.path.dirname(full_path))
    torch.save(model, full_path)

def load_model(model: nn.Module, model_name: str, directory: str | None):
    full_path = _make_model_path(model, model_name, directory)
    torch.load(full_path)


In [9]:
resid_model = ResidualNetwork((256, 256), in_channels, 101).to(device)


In [10]:
optimizer = torch.optim.Adam(resid_model.parameters(), lr=1e-3)
logger = CompositeLogger(JSONLogger('food100_conv_baseline.json'), StandardOutputLogger(100))

training_loop = TrainingLoop(resid_model, optimizer, loss_fn, logger)
# training_loop.fit(train_dataloader, test_dataloader, 5)
# save_model(resid_model, 'food100_conv_baseline', 'data/models')

## GAN

Let's make anime faces.

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [12]:
class AnimeDataset:
    def __init__(self, image_dir, transform=None, max_items=None):
        self.img_labels = list(os.listdir(image_dir))
        self.img_dir = image_dir
        self.transform = transform
        self._max_items = max_items

    def __len__(self):
        if self._max_items is None:
            return len(self.img_labels)
        return self._max_items

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels[idx])
        image = torchvision.io.decode_image(img_path)
        if self.transform:
            image = self.transform(image)
        return image

anime_dataset = AnimeDataset(
    'data/anime_faces/images',
    transform=Compose([
        torchvision.transforms.ConvertImageDtype(torch.float32),
        torchvision.transforms.Normalize((.5, .5, .5), (.5, .5, .5)),
        Resize((64, 64))]
    ),
    max_items=None)

anime_loader = DataLoader(anime_dataset, 64, shuffle=True)


FileNotFoundError: [Errno 2] No such file or directory: 'data/anime_faces/images'

In [ ]:
# custom weights initialization https://docs.pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
def _model_path(path, index):
    return f'{path}_{index}.pth'

def save_tuned_model(loop: GANLoop, path: str, index: int, meta: dict | None = None):
    directory, name = os.path.split(path)
    os.makedirs(directory, exist_ok=True)
    full_path = _model_path(path, index)
    loop.save_checkpoint(full_path, meta)

def load_tuned_model(path, index) -> tuple[GANLoop, dict]:
    full_path = _model_path(path, index)
    loop, meta = GANLoop.load_checkpoint(full_path)
    return loop, meta

def plot_loop_losses(g_loss, d_loss):
    fig, ax = plt.subplots(1, 1, figsize=(6, 3))
    ax.plot(range(len(g_loss)), g_loss, label='generator loss')
    ax.plot(range(len(d_loss)), d_loss, label='discriminator loss')
    fig.legend()


In [ ]:
def build_gan_loop(learning_rate: float, n_generator_features: int, n_discriminator_features: int, generator_input_size: int = 100, dropout: float = 0.) -> GANLoop:
    generator = Generator(generator_input_size, n_generator_features, dropout=dropout)
    discriminator = Discriminator(n_discriminator_features, dropout=dropout)
    generator.apply(weights_init)
    discriminator.apply(weights_init)

    loss = nn.BCEWithLogitsLoss()
    loop = GANLoop(
        generator,
        discriminator,
        torch.optim.Adam(generator.parameters(), lr=learning_rate, betas=(.5, .999)),
        torch.optim.Adam(discriminator.parameters(), lr=learning_rate, betas=(.5, .999)),
        loss
    )
    return loop


class TuneGANModel:
# TODO: TuneGANModel is close to a generic HP tuner.
#       Not implemented is a validation set for learning on
#       labeled data. Add that, then move this to nn_loop.
    def __init__(self, build_loop: Callable[..., GANLoop], paramgrid: dict):
        self.build = build_loop
        self.paramgrid = paramgrid

    def select_parameters(self, n_combinations: int) -> list[dict]:
        """Return n randomly selected hyperparameter combinations from the parameter grid."""
        parameters_array = np.array(list((itertools.product(*self.paramgrid.values()))))
        parameters_indexes = np.random.choice(
            np.arange(0, parameters_array.shape[0], step=1, dtype=np.int64),
            min(n_combinations, parameters_array.shape[0]),
            replace=False)
        selected_parameters = parameters_array[parameters_indexes]
        return [self._to_parameter_dict(params) for params in selected_parameters]

    def _to_parameter_dict(self, parameters):
        """Map parameters to parameter names and conform type to self.build annotations."""
        build_parameters = inspect.signature(self.build).parameters
        type_dict = {name: param.annotation for name, param in build_parameters.items() if type(param.annotation) is type}
        out = {}
        for param_name, param_value in zip(self.paramgrid, parameters):
            param_type = type_dict.get(param_name, type(param_value))
            out[param_name] = param_type(param_value)
        return out

    def tune_model(self, data: DataLoader, n_models: int, epochs: int, max_steps: int, callbacks: list | None):
        selected_parameters = self.select_parameters(n_models)
        for parameters in selected_parameters:
            print(parameters)
            loop = self.build(**parameters)

            callbacks = callbacks if callbacks is not None else [
                EndOnBatchCallback(max_steps),
                VerboseTrainingCallback(max_steps // 4),
                SnapshotCallback(8, interval=max_steps // 4)
            ]

            loop.fit(data, epochs, callbacks)


In [ ]:
class TestTuneGANMOdel:
    def test_select_params(self):
        model = TuneGANModel(build_gan_loop, {'dropout': [.01, .2], 'learning_rates': [.01, .0001], 'generator_input_size': [1, 2]})
        selected = model.select_parameters(2)
        assert len(selected) == 2
        assert 'dropout' in selected[0] and 'learning_rates' in selected[0]
        gen_size_type = type(selected[0]['generator_input_size'])
        assert gen_size_type is int, f"Expected int, but it is {gen_size_type}"

TestTuneGANMOdel().test_select_params()


In [ ]:
learning_rates = np.logspace(-3, -4.5, num=20)

parameter_grid = {
    'learning_rate': learning_rates.tolist(),
    'n_generator_features': [36, 40, 44, 48, 60],
    'n_discriminator_features': [36, 40, 44, 48, 60],
    'generator_input_size': [100, 150]
}

gan_tuner = TuneGANModel(build_gan_loop, parameter_grid)

In [ ]:
gan_tuner.tune_model(anime_loader, 3, 1, 10, None)

In [ ]:
# I think all this is deprecated.

# def tune_learning_rate(rates: list, n_features: int, n_combinations: int, max_steps: int, epochs: int = 1):

#     results = {}
#     max_steps = min(len(anime_dataset), max_steps)
#     snapshot_interval = max_steps // 8 if max_steps is not None else None
#     for idx in parameters:
#         learning_rate, n_gen_features, n_dis_features = parameters[idx]
#         dis_rate = learning_rate
#         print(
#             f"Starting with {idx} generator rate {learning_rate}, discriminator rate {dis_rate}" +
#             f"generator features: {n_gen_features} discriminator features: {n_dis_features}"
#         )
#         random_inputs = torch.randn((8, generator.input_size, 1, 1)).to(device)
#         input_one = random_inputs[0].view(1, generator.input_size, 1, 1)

#         images = SnapshotCallback(8, 'epoch_end', 1)
#         snapshots = SnapshotCallback(1, 'batch_end', max_steps // 8)
#         callbacks = [VerboseTrainingCallback(max_steps // 4), snapshots, images]

#         if max_steps:
#             callbacks.append(EndOnBatch(max_steps))

#         loop.fit(anime_loader, epochs, callbacks)
#         selected_parameters = {'gen_rate': learning_rate, 'dis_rate': dis_rate, 'gen_features': n_gen_features, 'dis_features': n_dis_features}
#         results[idx] = {'parameters': selected_parameters, 'images': images.snapshots, 'snapshots': snapshots.snapshots}
#         save_tuned_model(loop, 'gan_tuning/model', idx, results[idx])
#         show_image(make_grid(images.snapshots[-1].to('cpu'), nrow=4))
#         plt.show()
#         plot_loop_losses(loop.history['gen_loss'], loop.history['dis_loss'])
#         plt.show()

#         print()
#     # capture.save('gan_tuning/generator_images.pth')
#     return results


# results = tune_learning_rate(learning_rates.tolist(), 10, 100)


In [ ]:
for k, r in results.items():
    print(k)
    show_image(make_grid(r['images'][0].to('cpu'), nrow=4))
    show_image(make_grid(torch.concat(r['snapshots']).to('cpu')))

In [ ]:
{int(k): (float(v['gen_rate']), v['dis_rate']) for k, v in results.items()}

In [ ]:
selected_loop = 112
try:
    loop, meta = GANLoop.load_checkpoint((f'well_tuned_{selected_loop}.pth'))
except FileNotFoundError:
    print("Not found.")
    loop, meta = load_tuned_model('gan_tuning/model', selected_loop)
random_inputs = torch.randn((8, loop.generator.input_size, 1, 1)).to(device)


In [ ]:
epochs = 20
for epoch in range(epochs):
    show_image(make_grid(loop.generator(random_inputs).cpu()))
    step = 0
    print(f"Epoch: {epoch}")
    for gen_loss, dis_loss in loop.step(anime_loader):
        if step % (len(anime_dataset) // anime_loader.batch_size // 5) == 0:
            print(f"Gen loss: {gen_loss}, dis loss: {dis_loss}")
        step += 1

show_image(make_grid(loop.generator(random_inputs).cpu()))


In [ ]:
loop.save_checkpoint(f'well_tuned_{selected_loop}.pth', meta)

In [ ]:
show_image(loop.generator(random_inputs.to(device)).cpu()[-4])

In [ ]:
def report_result(result: dict):
    x = np.arange(0, len(result['gen_loss']))
    plt.plot(x, result['gen_loss'], label='generator loss')
    plt.plot(x, result['dis_loss'], label='dis_loss')

report_result(meta)